# Support Scripts

This notebook is a collection of scripts which are useful in administering your portal instance.

## Get Time Played by User
Run this script to get the total amount of time spent logged in by payer. Change @sinceDateTime to the earliest datetime that you want to measure from.

In [ ]:
DECLARE @sinceDate DATETIME = '2020-03-18 21:00';

DROP TABLE IF EXISTS #timePlayed;

CREATE TABLE #timePlayed 
(
	Id INT Identity(1,1) NOT NULL PRIMARY KEY CLUSTERED,
	LoggedIn DATETIME,
	LoggedOut DATETIME,
	GamerTag VARCHAR(50),
	MsLoggedIn AS DATEDIFF_BIG(MILLISECOND, LoggedIn, LoggedOut)
);

INSERT INTO #timePlayed (GamerTag, LoggedIn)
SELECT DISTINCT 
	SUBSTRING(LogText, CHARINDEX('connected: ', LogText) + 11, (CHARINDEX(', xuid', LogText)) - (CHARINDEX('connected: ', LogText) + 11)),
	LogTime
FROM dbo.ApplicationLog 
WHERE LogTime >= @sinceDate
	  AND LogText LIKE '%Player connected%'
ORDER BY LogTime DESC;



UPDATE #timePlayed
SET LoggedOut = (
			SELECT TOP (1) LogTime 
			FROM dbo.ApplicationLog al
			WHERE al.LogTime > #timePlayed.LoggedIn
			  AND al.LogText LIKE '%disconnected: ' + #timePlayed.GamerTag + '%'
			ORDER BY LogTime 
		);


SELECT
	tp.GamerTag,
	anu.DiscordId,
	CAST(SUM(MsLoggedIn) / 3.6e+6 AS DECIMAL(27,2)) AS HoursPlayed
FROM #timePlayed tp
    JOIN dbo.AspNetUsers anu ON anu.GamerTag = tp.GamerTag
GROUP BY tp.GamerTag, anu.DiscordId
ORDER BY HoursPlayed DESC;

DROP TABLE #timePlayed;

## Get Days Since Logon

Get the number of days since each user last logged onto the Minecraft server.

In [ ]:
DECLARE @now DATETIME = GETUTCDATE();

SELECT  
	GamerTag,
	DATEDIFF(DAY, LastMinecraftLogin, @now) AS DaysSinceLoggedIn
FROM AspNetUsers
WHERE LastMinecraftLogin IS NOT NULL
ORDER BY LastMinecraftLogin DESC;